In [0]:
from pyspark.sql.functions import *

In [0]:
from pyspark.sql.types import *
from datetime import datetime

data = [( 1, datetime.strptime('2011-01-01', '%Y-%m-%d'), 500),
        ( 1, datetime.strptime('2011-01-15', '%Y-%m-%d'), 50),
        ( 1, datetime.strptime('2011-01-22', '%Y-%m-%d'), 250),
        ( 1, datetime.strptime('2011-01-24', '%Y-%m-%d'), 75),
        ( 1, datetime.strptime('2011-01-26', '%Y-%m-%d'), 125),
        ( 1, datetime.strptime('2011-01-28', '%Y-%m-%d'), 175),
        ( 2, datetime.strptime('2011-01-01', '%Y-%m-%d'), 500),
        ( 2, datetime.strptime('2011-01-15', '%Y-%m-%d'), 50),
        ( 2, datetime.strptime('2011-01-22', '%Y-%m-%d'), 25),
        ( 2, datetime.strptime('2011-01-23', '%Y-%m-%d'), 125),
        ( 2, datetime.strptime('2011-01-26', '%Y-%m-%d'), 200),
        ( 2, datetime.strptime('2011-01-29', '%Y-%m-%d'), 250),
        ( 3, datetime.strptime('2011-01-01', '%Y-%m-%d'), 500),
        ( 3, datetime.strptime('2011-01-15', '%Y-%m-%d'), 50 ),
        ( 3, datetime.strptime('2011-01-22', '%Y-%m-%d'), 5000),
        ( 3, datetime.strptime('2011-01-25', '%Y-%m-%d'), 550),
        ( 3, datetime.strptime('2011-01-27', '%Y-%m-%d'), 95 ),
        ( 3, datetime.strptime('2011-01-30', '%Y-%m-%d'), 2500)]

my_schema = StructType([
    StructField("AccountId", IntegerType(), False),
    StructField("TranDate", DateType(), False),
    StructField("TranAmt", IntegerType(), False)
])

Transactions = spark.createDataFrame(data, schema=my_schema)
display(Transactions)

AccountId,TranDate,TranAmt
1,2011-01-01,500
1,2011-01-15,50
1,2011-01-22,250
1,2011-01-24,75
1,2011-01-26,125
1,2011-01-28,175
2,2011-01-01,500
2,2011-01-15,50
2,2011-01-22,25
2,2011-01-23,125


In [0]:
data = [(1,'George', 800),
        (2,'Sam', 950),
        (3,'Diane', 1100),
        (4,'Nicholas', 1250),
        (5,'Samuel', 1250),
        (6,'Patricia', 1300),
        (7,'Brian', 1500),
        (8,'Thomas', 1600),
        (9,'Fran', 2450),
        (10,'Debbie', 2850),
        (11,'Mark', 2975),
        (12,'James', 3000),
        (13,'Cynthia', 3000),
        (14,'Christopher', 5000)]

my_schema = StructType([
    StructField('RowID', IntegerType(), False),
    StructField('FName', StringType(), False),
    StructField('Salary', IntegerType(), False)
])

logical = spark.createDataFrame(data, schema=my_schema)
display(logical)

RowID,FName,Salary
1,George,800
2,Sam,950
3,Diane,1100
4,Nicholas,1250
5,Samuel,1250
6,Patricia,1300
7,Brian,1500
8,Thomas,1600
9,Fran,2450
10,Debbie,2850


In [0]:
from pyspark.sql.window import Window

window_spec = Window.partitionBy('AccountId').orderBy('TranDate')

part_df = Transactions.withColumn(
    'RunTotalAmt', sum("TranAmt").over(window_spec)).withColumn(
    'RunAvg', avg("TranAmt").over(window_spec)).withColumn(
    'RunTranQty', count("*").over(window_spec)).withColumn(
    'RunSmallAmt', min('TranAmt').over(window_spec)).withColumn(
    'RunLargeAmt', max('TranAmt').over(window_spec)).orderBy('AccountId', 'TranDate')
    

display(part_df)

AccountId,TranDate,TranAmt,RunTotalAmt,RunAvg,RunTranQty,RunSmallAmt,RunLargeAmt
1,2011-01-01,500,500,500.0,1,500,500
1,2011-01-15,50,550,275.0,2,50,500
1,2011-01-22,250,800,266.6666666666667,3,50,500
1,2011-01-24,75,875,218.75,4,50,500
1,2011-01-26,125,1000,200.0,5,50,500
1,2011-01-28,175,1175,195.83333333333334,6,50,500
2,2011-01-01,500,500,500.0,1,500,500
2,2011-01-15,50,550,275.0,2,50,500
2,2011-01-22,25,575,191.66666666666666,3,25,500
2,2011-01-23,125,700,175.0,4,25,500


In [0]:
window_spec = Window.partitionBy('AccountId').orderBy('TranDate').rowsBetween(-2, 0)
window_spec_rn = Window.partitionBy('AccountId').orderBy('TranDate')

sec_part_df = Transactions.withColumn(
    'SlideAvg', avg('TranAmt').over(window_spec)).withColumn(
        'SlideQty', count('*').over(window_spec)).withColumn(
            'SlideMin', min('TranAmt').over(window_spec)).withColumn(
                'SlideMax', max('TranAmt').over(window_spec)).withColumn(
                    'SlideTotal', sum('TranAmt').over(window_spec)).withColumn(
                        'Row Number', row_number().over(window_spec_rn)).orderBy('AccountId', 'TranDate')

display(sec_part_df)

AccountId,TranDate,TranAmt,SlideAvg,SlideQty,SlideMin,SlideMax,SlideTotal,Row Number
1,2011-01-01,500,500.0,1,500,500,500,1
1,2011-01-15,50,275.0,2,50,500,550,2
1,2011-01-22,250,266.6666666666667,3,50,500,800,3
1,2011-01-24,75,125.0,3,50,250,375,4
1,2011-01-26,125,150.0,3,75,250,450,5
1,2011-01-28,175,125.0,3,75,175,375,6
2,2011-01-01,500,500.0,1,500,500,500,1
2,2011-01-15,50,275.0,2,50,500,550,2
2,2011-01-22,25,191.66666666666666,3,25,500,575,3
2,2011-01-23,125,66.66666666666667,3,25,125,200,4


In [0]:
window_rows = Window.orderBy("Salary").rowsBetween(Window.unboundedPreceding, Window.currentRow)
window_range = Window.orderBy("Salary").rangeBetween(Window.unboundedPreceding, Window.currentRow)

df = logical.withColumn("SumByRows", sum("Salary").over(window_rows)) \
       .withColumn("SumByRange", sum("Salary").over(window_range))

display(df.orderBy('RowID'))

RowID,FName,Salary,SumByRows,SumByRange
1,George,800,800,800
2,Sam,950,1750,1750
3,Diane,1100,2850,2850
4,Nicholas,1250,4100,5350
5,Samuel,1250,5350,5350
6,Patricia,1300,6650,6650
7,Brian,1500,8150,8150
8,Thomas,1600,9750,9750
9,Fran,2450,12200,12200
10,Debbie,2850,15050,15050
